# REPORT

## 1. Methodology, approach and model selection rational

Lending club dataset contains two types of information: the one that was available after loan application submission and the one that was collected after the date of loan issue. As our goal is to make decision on loan issue based on forecast on default status, first thing I did was to identify those features on our dataset, that might be available before loan was issued.

Our dataset has 2 types of applications - individual and joint applications, and set of features related to second applicant or joint features, that are not filled in most of the cases, i.e. for individual applications. For this reason I made a new set of adjusted features that replaces individual and second applicant features, i.e. takes sum or average of both individual and second applicant features or just the total feature, depending on the case. In addition I made target feature `default status` that is `True` in case the current status contains `Charged Off`.

In order to select most important features for the model, I've chosen features with correlation rate to target more than 5% and skipped ones of those that has intercorrelation rate between each other more than 75%. That gave me 18 most important features for my model. 

![Target Correlation](../img/target_corr.png)
![Inter Correlation](../img/feat_corr_heatmap.png)

Then I split dataset to train/test datasets and chained in a pipeline steps for **(i)** preprocessing chosen features such as encoding of categorical features, imputing missing values, standardizing numerical features, **(ii)** handling imbalanced data by applying SMOTE and **(iii)** fitting/predicting data with chosen regressor. 

For this project I choose to use **RandomForestClassifier** as a baseline model and **CatBoostClassifier** as candidate model. For models' evaluation I calculated actual profit rate for non-defaulted loans and actual loss rate for defaulted loans. Applying range of probability thresholds for models probability predictions I calculated confusion matrices for each threshold looking for minimized losses. CatBoost gave slightly better results in terms of minimizing losses with probability threshold 42%.

Although my both models gave relatively low precisions (0.36 - 0.39) and ROC_AUC (0.52-0.55) scores, I believe further hyperparameter's tuning would help to increase those metrics if I'd have more time fro submission.


## 2. Advantages and limitations of the chosen model

Both CatBoost and RandomForest are decision tree ensemble Machine Learning algorithms, that means they're using smaller trees datasets to make predictions. CatBoost uses boost techniques that generates trees one after another, where the results of each tree improves results of next tree. RandomForest split original dataset randomly on a smaller trees to make better predictions on each individual tree, than generalize them in one more accurate prediction.

Advantages of CatBoost model:
  - works much better with categorical data 
  - ability to use GPU for faster fitting and predictions

Disadvantages comparing to RandomForest:
  - much harder to tune
  - trees can't be build in parallel

## 3. Architecture and final solution, considerations on deployment

Simple architecture schema

![Architecture](../img/Architecture.png)

Final solution might be implemented as Kedro project as a package or Docker container. Model training and hypertuning on historical dataset could be done using advantages of Kedro Viz package that has an ability to trace model's results, that could be useful in future as well to retrain the model on new dataset, when it will be available.

After model tuning one more pipeline could be added to Kedro project that will take new data files in data/1-raw subfolder, preprocess it to proper data types, add new features and make prediction using model pipeline. Then build a web application within Kedro project that will receive new datasets from User and make predictions using pipeline described above. Deploy it on Backend server of the Company.


## 4. Estimated impact

Implementation of my model will reduce losses of the business from defaults by $11.57 mln and reduce Non-Performing ratio from 0.05729 to 0.04974.